## Module 3 — Visualizing Data

There are a number of ways to visualize annotations and analysis of EM data. This tutorial will go into a few aspects that are more specific to working with the MICrONs EM data.

For general purpose python plotting, there are a number of useful tutorials out there. In particular, I recommend looking at [Seaborn](https://seaborn.pydata.org/), which makes it easy to work with data tables in the powerful visualization library [MatPlotLib](https://matplotlib.org/stable/tutorials/index.html). We would be happy to discuss such plotting, but will not go into it further here.

### Putting data into Neuroglancer

We have build a tool called `nglui` to take arrays of data and generate neuroglancer states with certain neurons selected, annotation points located, etc.

Statebuilder is a submodule that helps produce custom Neuroglancer states based on data in the form of Pandas dataframes. Neuroglancer organizes data sources into layers. Layers come in three types, image, segmentation, and annotation. Each has different properties and functions. The state builder lets the user define a set of rules for initializing layers and how to map data columns to selections and annotations. Let's see the simplest example in use now.



In [1]:
from caveclient import CAVEclient
from nglui import statebuilder

client = CAVEclient('minnie65_public')

### Layers and Statebuilders

The design of the statebuilder package follows the design of Neuroglancer. You specify the "rules" for each layer 

Image layers in Neuroglancer are pretty simple. An image has a name (by default 'img' here) and a source, which is typically a path to a cloud hosted 3d image volume. We define an image layer with an ImageLayerConfig that lets the user set these key parameters. Conveniently, we don't need to remember the path name and we can get it from the client instead.

Segmentation Layers just need the cloud path, which again we can get from the client.

In [2]:
img = statebuilder.ImageLayerConfig(
    source=client.info.image_source(),
    contrast_controls=True, # this just puts in the code needed to manually adjust contrast
)

seg = statebuilder.SegmentationLayerConfig(
    source=client.info.segmentation_source(),
)

Now we can assemble these layers together to a single StateBuilder:

In [3]:
sb = statebuilder.StateBuilder(
    layers=[img, seg],
    client=client,
)

At its simplest, you can just render out a link this way:

In [4]:
sb.render_state()

'https://neuroglancer.neuvue.io#!%7B%22jsonStateServer%22:%22https://global.daf-apis.com/nglstate/api/v1/post%22,%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4.0,4.0,40.0%5D%7D%7D,%22zoomFactor%22:2.0%7D,%22showSlices%22:false,%22layout%22:%22xy-3d%22,%22perspectiveZoom%22:2000.0,%22layers%22:%5B%7B%22type%22:%22image%22,%22source%22:%22precomputed://https://bossdb-open-data.s3.amazonaws.com/iarpa_microns/minnie/minnie65/em%22,%22name%22:%22img%22,%22shader%22:%22#uicontrol%20float%20black%20slider(min=0,%20max=1,%20default=0.0)%5Cn#uicontrol%20float%20white%20slider(min=0,%20max=1,%20default=1.0)%5Cnfloat%20rescale(float%20value)%20%7B%5Cn%20%20return%20(value%20-%20black)%20/%20(white%20-%20black);%5Cn%7D%5Cnvoid%20main()%20%7B%5Cn%20%20float%20val%20=%20toNormalized(getDataValue());%5Cn%20%20if%20(val%20%3C%20black)%20%7B%5Cn%20%20%20%20emitRGB(vec3(0,0,0));%5Cn%20%20%7D%20else%20if%20(val%20%3E%20white)%20%7B%5Cn%20%20%20%20emitRGB(vec3(1.0,%201.0,%201.0))

That's a bit a bit ugly, let's make it more pleasant:

In [5]:
sb.render_state(return_as='html')

# Data-driven links

Great, but now let's make it responsive to data!

In [6]:
# First, let's download some cell types
ct_df = client.materialize.tables.allen_v1_column_types_slanted_ref().query(
    desired_resolution=[4,4,40],  # Useful to ensure you know the resolution of the data points
)

# Glance at the columns of the table
ct_df.head()

The `client.materialize.tables` interface is experimental and might experience breaking changes before the feature is stabilized.


,id_ref,created_ref,valid_ref,target_id,classification_system,cell_type,id,created,valid,volume,pt_supervoxel_id,pt_root_id,pt_position,bb_start_position,bb_end_position
0,50,2023-03-18 14:13:21.613360+00:00,t,258319,aibs_coarse_excitatory,23P,258319,2020-09-28 22:40:42.476911+00:00,t,261.806162,89309001002848425,864691135698459925,"[178400, 143248, 21238]","[nan, nan, nan]","[nan, nan, nan]"
1,1119,2023-03-18 14:13:22.506660+00:00,t,276438,aibs_coarse_excitatory,6P-CT,276438,2020-09-28 22:40:42.700226+00:00,t,277.317714,89465269428261699,864691136487559186,"[179648, 258768, 23597]","[nan, nan, nan]","[nan, nan, nan]"
2,35,2023-03-18 14:13:21.602813+00:00,t,260552,aibs_coarse_excitatory,23P,260552,2020-09-28 22:40:42.745779+00:00,t,230.111805,89170256379033022,864691136040432126,"[177408, 157968, 21002]","[nan, nan, nan]","[nan, nan, nan]"
3,95,2023-03-18 14:13:21.644304+00:00,t,260263,aibs_coarse_excitatory,23P,260263,2020-09-28 22:40:42.746658+00:00,t,274.324193,88044356338331571,864691136334881203,"[169440, 158128, 20266]","[nan, nan, nan]","[nan, nan, nan]"
4,81,2023-03-18 14:13:21.634505+00:00,t,262898,aibs_coarse_inhibitory,BPC,262898,2020-09-28 22:40:42.749245+00:00,t,230.092308,88468836747612860,864691135654475970,"[172512, 175280, 21964]","[nan, nan, nan]","[nan, nan, nan]"


In [7]:
# We can slice this to get cells whose cell type is "5P-PT"

l5pt_df = ct_df[
    ct_df['cell_type'] == "5P-PT"
]

Now let's build the layers again. As always, we need an image and a segmentation layer, but there is a convenience function to get those from a client directly:

In [8]:
img, seg = statebuilder.from_client(client)

Now we want to add an annotation layer that knows how to transform the dataframe we have into neuroglancer point annotations. A typical annotation layer looks like the following:

* A name specifies the layer in the neuroglancer tabs
* A "linked segmentation layer" tells the annotation layer what segmentation layer to use to load root ids. This is specified by providing the name of the segmentation layer ("seg" by default), but is easiest done by using the `seg.name` property of the segmentation layer we just created.
* A "mapping rule" that describes how to map dataframe columns to annotation properties. `PointMapper` creates points annotations, a `LineMapper` creates line annotations, etc. In this case, we assign the point column (i.e. the location of the point) and the "linked segmentation" (i.e. the root id of the cell that will be highlighted).
* A data resolution that says how to interpret the points in nanometers. i.e. [4,4,40] indicates the points correspond to 4x4x40 nm voxels. For this dataset, most database values are in [4,4,40] resolution and this is treated as a default resolution (information stored in the dataset metadata) if a data resolution is omitted.

In [9]:
anno = statebuilder.AnnotationLayerConfig(
    name = 'my_cells',
    linked_segmentation_layer=seg.name,
    mapping_rules=statebuilder.PointMapper(
        point_column='pt_position',
        linked_segmentation_column='pt_root_id',
    ),
    data_resolution=[4,4,40]   # This should be the same as the desired_resolution above.
)

Now we build the statebuilder just like before, but with one more layer:

In [10]:
sb = statebuilder.StateBuilder(
    layers = [img, seg, anno],
    client=client,
)

Now we can render the data:

In [11]:
sb.render_state(return_as='html')

Wait, this looks no different! That's because we only gave the statebuilder the rules, but not the data.

In [12]:
sb.render_state(
    data=l5pt_df,
    return_as='html',
)

Aha, here we are! Now we can see the points we asked for!

But even better, this statebuilder just has _rules_, not data. So now we could pass different data without changing anything other than the dataframe we provide. For example, let's pick out inhibitory cells, which have the value `aibs_coarse_inhibitory` in the column `classification_system`.

In [13]:
sb.render_state(
    data=ct_df[ ct_df['classification_system'] == "aibs_coarse_inhibitory"],
    return_as='html',
)

## Adding tags

Pre-specified tags are useful for labeling data. We can pre-specify the tag options available in the links by adding a `tags` list of arguments.

In [18]:
anno_w_tags = statebuilder.AnnotationLayerConfig(
    name = 'my_cells',
    linked_segmentation_layer=seg.name,
    mapping_rules=statebuilder.PointMapper(
        point_column='pt_position',
        linked_segmentation_column='pt_root_id',
    ),
    tags=['nice', 'naughty'],
)

Now we can use this in a statebuilder, and you can see that the annotation label

In [20]:
sb = statebuilder.StateBuilder(
    layers = [img, seg, anno_w_tags],
    client=client,
)

sb.render_state(
    data=ct_df[ ct_df['classification_system'] == "aibs_coarse_inhibitory"],
    return_as='html',
)

## Further documentation

There are a lot of additional control one has over neuroglancer states. Find more complete [documentation here](https://github.com/seung-lab/NeuroglancerAnnotationUI/blob/master/examples/statebuilder_examples.ipynb).